In [1]:
import gzip
def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

all_ratings = []
for user,book,r in readCSV("train_Interactions.csv.gz"):
  all_ratings.append([user,book,float(r)])
  

In [90]:
from surprise import SVDpp, accuracy
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
import numpy as np
from surprise.model_selection.split import KFold
from collections import defaultdict

reader = Reader(line_format='user item rating', sep=',', skip_lines=1)
data = Dataset.load_from_file("" + "train_Interactions.csv", reader=reader)
trainset, testset = train_test_split(data, test_size=.25)

# data = Dataset.load_from_df(df_ratings, reader=reader)
bestMSE = 2
best_i = 0
best_u = 0

i_mse = defaultdict(list)
u_mse = defaultdict(list)

for i in [0,1]:
    for u in [0.14,0.11, 0.12,0.13,0.02]:

        kf = KFold(n_splits=5)

        accur = []

        for trainset, testset in kf.split(data):

    # train and test algorithm.
            model = SVDpp(n_factors = i, reg_all = u)
            model.fit(trainset)
            predictions = model.test(testset)

    # Compute and print Root Mean Squared Error
            accur.append(accuracy.mse(predictions, verbose=False))

        accur = np.array(accur)
    
        mse_current = np.mean(accur)
        if(mse_current < bestMSE):
            best_i = i
            best_u = u
            
            bestMSE = mse_current

        print("reg_all = ", u, " n_factors = ", i, "MSE = ", mse_current)
        i_mse[i].append(mse_current)
        u_mse[u].append(mse_current)

print("bestMSE = ", bestMSE)
print("best parameter n factor = ", best_i, " reg = ", best_u)




reg_all =  0.14  n_factors =  0 MSE =  1.4559047720115021
reg_all =  0.11  n_factors =  0 MSE =  1.453819628817556
reg_all =  0.12  n_factors =  0 MSE =  1.452639008455994
reg_all =  0.13  n_factors =  0 MSE =  1.453475874510454
reg_all =  0.02  n_factors =  0 MSE =  1.4575371623908342
reg_all =  0.14  n_factors =  1 MSE =  1.452300258490642
reg_all =  0.11  n_factors =  1 MSE =  1.4541993770297137
reg_all =  0.12  n_factors =  1 MSE =  1.4531866081373106
reg_all =  0.13  n_factors =  1 MSE =  1.455387113128535
reg_all =  0.02  n_factors =  1 MSE =  1.463199911374528
bestMSE =  1.452300258490642
best parameter n factor =  1  reg =  0.14


In [91]:
avg_i = defaultdict(float)
avg_u = defaultdict(float)
for i in i_mse:
    avg_i[i] = sum(i_mse[i]) / len(i_mse[i])

for u in u_mse:
    avg_u[u] = sum(u_mse[u]) / len(u_mse[u])

print(dict(sorted(avg_i.items(), key=lambda item: item[1])))
print(dict(sorted(avg_u.items(), key=lambda item: item[1])))




{0: 1.4546752892372683, 1: 1.455654653632146}
{0.12: 1.4529128082966523, 0.11: 1.4540095029236348, 0.14: 1.454102515251072, 0.13: 1.4544314938194944, 0.02: 1.460368536882681}


In [94]:
trainset, testset = train_test_split(data, test_size=0.000000001)
model = SVDpp(reg_all=0.11,n_factors=0)
model.fit(trainset)

In [132]:
trainset

In [128]:
from collections import defaultdict
userIDs = defaultdict(int)
itemIDs = defaultdict(int)
for d in trainset:
    u = d[0]
    i = d[1]
    if(u not in userIDs):
        userIDs[u] = len(userIDs)
    if(i not in itemIDs):
        itemIDs[i] = len(itemIDs)

len(userIDs)
len(itemIDs)

TypeError: 'Trainset' object is not iterable

In [59]:
# LFM using tensorflow
import random
import tensorflow as tf
from collections import defaultdict

userIDs = defaultdict(int)
itemIDs = defaultdict(int)
interactions = []

for d in all_ratings:
    u = d[0]
    i = d[1]
    r = d[2]
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))

random.shuffle(interactions)


nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)


class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)


def trainingStep(model, interactions_ts):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions_ts)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer = tf.keras.optimizers.Adam(0.1)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()


for k in [1,2,3,4,5,6,10,50]:
    modelLFM = None
    modelLFM = LatentFactorModel(mu, k, 20)
    for i in range(100):
        obj = trainingStep(modelLFM, interactionsTrain)
        # if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

    MSE = 0
    for (u,i,r) in interactionsTest:
        # u,i,r = interactionsTest[0]
        predict = modelLFM.predict(userIDs[u], itemIDs[i]).numpy()
        MSE += (predict - r) ** 2
    MSE /= len(interactionsTest)

    print("MSE = ", MSE, "for k = ", k)

MSE =  1.7256470083265885 for k =  1
MSE =  1.7301639316147357 for k =  2
MSE =  1.725289218377765 for k =  3
MSE =  1.7252584805389717 for k =  4
MSE =  1.7295875615159824 for k =  5
MSE =  1.725320719379112 for k =  6
MSE =  1.7264886989580035 for k =  10
MSE =  1.7257932110429341 for k =  50


In [73]:
modelLFM = LatentFactorModel(mu, 4, 20)
def getMSE_lfm():
    MSE = 0
    for (u,i,r) in interactionsTest:
        # u,i,r = interactionsTest[0]
        predict = float(modelLFM.predict(userIDs[u], itemIDs[i]))
        MSE += (predict - r) ** 2
    MSE /= len(interactionsTest)
    return MSE


has_improved = True
pre_ite_mse = getMSE_lfm()
post_ite_mse = 0

iteraction = 0
while(iteraction < 100):
    obj = trainingStep(modelLFM, interactions)
    # post_ite_mse = getMSE_lfm()
    # if(pre_ite_mse - post_ite_mse <= 0):
    #     has_improved = False
    # pre_ite_mse = post_ite_mse
    iteraction += 1


In [74]:
getMSE_lfm()

1.7253735009883755

In [35]:
MSE = 0
for (u,i,r) in interactionsTest:
        # u,i,r = interactionsTest[0]
    predict = float(modelLFM.predict(userIDs[u], itemIDs[i]))
    MSE += (predict - r) ** 2
MSE /= len(interactionsTest)
print("MSE = ", MSE)

MSE =  0.6489271233778804


In [93]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("./pairs_Rating.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    # Logic...
    
    # _ = predictions.write(u + ',' + g + ',' + str(predict_rating(u,g)) + '\n')

    prediction = model.predict(u,g).est
    # prediction = float(modelLFM.predict(userIDs[u], itemIDs[i]))

    # prediction = modelLFM.predict(userIDs[u], itemIDs[i]).numpy()
    # print(prediction)
    if(prediction < 0):
        prediction = 0
    if(prediction > 5):
        prediction = 5


    _ = predictions.write(u + ',' + g + ',' + str(prediction) + '\n')

predictions.close()

In [34]:
float(modelLFM.predict(2, 1))


3.8329050540924072

In [120]:
import random
from collections import defaultdict
random.shuffle(all_ratings)
cutoff_index = int(len(all_ratings) * 0.2)
trainset = all_ratings[:cutoff_index]
testset = all_ratings[cutoff_index:]

itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)

# for d in train_entries:
#     itemsPerUser[d[0]].append(d[1])
#     usersPerItem[d[1]].append(d[0])

In [123]:
#################################################
# Hours played prediction                        #
#################################################
from collections import defaultdict


trainRating = [float(r[2]) for r in trainset]
globalAverage = sum(trainRating) * 1.0 / len(trainRating)
 
### Question 6
betaU = model.bu
betaI = model.bi

ratings = defaultdict(float)
all_user_set = set()
all_item_set = set()

for d in trainset:
    item = d[1]
    user = d[0]
    ratings[(user, item)] = float(d[2])
    all_user_set.add(user)
    all_item_set.add(item)


alpha = globalAverage # Could initialize anywhere, this is a guess

def iterate(lamb):

    alpha = 0
    for allKeys in ratings:
        u = allKeys[0]
        i = allKeys[1]
        alpha += ratings[(u,i)] - (betaU[u] + betaI[i])
    alpha /= len(ratings)

    for user in all_user_set:
        betaU[user] = 0
        for i in itemsPerUser[user]:
            betaU[user] += ratings[(user, i)] - (alpha + betaI[i])
        betaU[user] /= lamb + len(itemsPerUser[user])

    for item in all_item_set:
        betaI[item] = 0
        for u in usersPerItem[item]:
            betaI[item] += ratings[(u, item)] - (alpha + betaU[u])
        betaI[item] /= lamb + len(usersPerItem[item])

    

def findLost(lamb):
    lost = 0
    for allKeys in testset:
        u = allKeys[0]
        i = allKeys[1]
        prediction = 0
        if(u not in betaU):
            if(i in betaI):
                prediction = alpha + betaI[i]
            else:
                prediction = alpha
        elif(i not in betaI):
            prediction = alpha + betaU[u]
        else:
            prediction =  alpha + betaU[u] + betaI[i]

        lost += (prediction - allKeys[2]) ** 2
    complexity = 0
    for u in betaU:
        complexity += betaU[u] ** 2
    for i in betaI:
        complexity += betaI[i] ** 2
    lost += lamb * complexity
    return lost

def predict_rating(predict_user, predict_item):
    if(predict_item not in betaI):
        if(predict_user in betaU):
            prediction = alpha + betaU[predict_user]
        else:
            prediction = alpha
    elif(predict_user not in betaU):
        prediction = betaI[predict_item]
    else:
        prediction = alpha + betaU[predict_user] + betaI[predict_item]
    
    return prediction
    

def find_mse_given_lamb(lamb):

    for u in itemsPerUser:
        betaU[u] = 0

    for g in usersPerItem:
        betaI[g] = 0

    converge = False
    count = 0
    while(converge == False):
        old_objective = findLost(lamb)
        iterate(lamb)
        new_objective = findLost(lamb)
        if(old_objective - new_objective < 0 ):
            converge = True
        count += 1
    validMSE = 0
    for each_val_item in valid_entries:
        actual_label = each_val_item[2]
        predict_user = each_val_item[0]
        predict_item = each_val_item[1]

        prediction = predict_rating(predict_user, predict_item)

        
        validMSE += (actual_label - prediction) ** 2
    validMSE /= len(valid_entries)

    return validMSE

    

### Question 7
# betaUs = [(betaU[u], u) for u in betaU]
# betaIs = [(betaI[i], i) for i in betaI]
# betaUs.sort()
# betaIs.sort()

# print("Maximum betaU = " + str(betaUs[-1][1]) + ' (' + str(betaUs[-1][0]) + ')')
# print("Maximum betaI = " + str(betaIs[-1][1]) + ' (' + str(betaIs[-1][0]) + ')')
# print("Minimum betaU = " + str(betaUs[0][1]) + ' (' + str(betaUs[0][0]) + ')')
# print("Minimum betaI = " + str(betaIs[0][1]) + ' (' + str(betaIs[0][0]) + ')')



best_lam = 0
best_mse = 10

# for l in range(1, 20):
#     lamb = l / 10
#     cur_mse = find_mse_given_lamb(lamb)
#     if(cur_mse < best_mse):
#         best_mse = cur_mse
#         best_lam = lamb


find_mse_given_lamb(1.0)

predictions = open("predictions_Rating.csv", 'w')
for l in open("./pairs_Rating.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    # Logic...
    
    # _ = predictions.write(u + ',' + g + ',' + str(predict_rating(u,g)) + '\n')

    prediction = model.predict(u,g).est
    # print(prediction)


    _ = predictions.write(u + ',' + g + ',' + str(prediction) + '\n')

predictions.close()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices